In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings 
warnings.filterwarnings("ignore") #ignore some matplotlib warnings

import numpy as np

from triqs.plot.mpl_interface import plt, oplot, oplotr, oploti
plt.rcParams["figure.figsize"] = (6,5) # set default size for all figures

# Lattice Bethe-Salpeter Equation (BSE)

First we setup a dispersion for the lattice.

This can be done using a tight-binding Hamiltonian, see the square lattice guide.

Here we will keep thing simple and perform a calculation using the self energy and the local vertex of the Hubbard atom. This example can be adapted to use the result of a general impurity problem, e.g. from `triqs/cthyb`.

In [2]:
from triqs_tprf.analytic_hubbard_atom import analytic_hubbard_atom

p = analytic_hubbard_atom(
    beta = 1.234,
    U = 5.0,
    nw = 1,
    nwf = 248,
    nwf_gf = 2 * 248,
    )

In [3]:
from triqs.lattice import BrillouinZone, BravaisLattice
bl = BravaisLattice([(1,0,0), (0,1,0)])
bz = BrillouinZone(bl)    

from triqs.gf import MeshBrZone
bzmesh = MeshBrZone(bz, n_k=1) # only one k-point

from triqs.gf import Gf
e_k = Gf(mesh=bzmesh, target_shape=[1, 1])
e_k *= 0.

For the Hubbard atom we can compute the local self energy $\Sigma(i\omega_n)$

In [4]:
Sigma_iw = p.G_iw.copy()
G0_iw = p.G_iw.copy()

from triqs.gf import inverse, iOmega_n
G0_iw << inverse(iOmega_n + 0.5*p.U)
Sigma_iw << inverse(G0_iw) - inverse(p.G_iw)

Green Function G_dn with mesh Matsubara Freq Mesh of size 992, Domain: Matsubara domain with beta = 1.234, statistic = Fermion, positive_only : 0 and target_rank 2: 

By using the atomic self energy we can compute the lattice Green's function (in the Hubbard I approximation)

In [5]:
from triqs_tprf.lattice import lattice_dyson_g_wk
g_wk = lattice_dyson_g_wk(mu=0.5*p.U, e_k=e_k, sigma_w=Sigma_iw)

from triqs_tprf.lattice import fourier_wk_to_wr
g_wr = fourier_wk_to_wr(g_wk)

With the Green's function we can construct the lattice bare susceptbility by making the product in imaginary time and real space using `triqs_tprf.lattice.chi0r_from_gr_PH` and `triqs_tprf.lattice.chi0q_from_chi0r`

In [6]:
from triqs_tprf.lattice import chi0r_from_gr_PH
chi0_wr = chi0r_from_gr_PH(nw=p.nw, nn=p.nwf, g_nr=g_wr) 

from triqs_tprf.lattice import chi0q_from_chi0r
chi0_wk = chi0q_from_chi0r(chi0_wr)

With $\chi_0$ and a local vertex from an impurity calculation (here we use the analytic Hubbard atom vertex) we can solve the Bethe-Salpeter equation on the lattice using `triqs_tprf.lattice.chiq_from_chi0q_and_gamma_PH`

In [7]:
from triqs_tprf.lattice import chiq_from_chi0q_and_gamma_PH
chi_wk = chiq_from_chi0q_and_gamma_PH(chi0_wk, p.gamma_m)

In [8]:
num = np.squeeze(chi_wk.data.real)
ref = np.squeeze(p.chi_m.data.real)

diff = np.max(np.abs(num - ref))
print('diff =', diff)

diff = 0.5861047169392135


There are also helper functions to sum over fermionic freqiencies in order to obtain the one frequency dependent generalized susceptibility

In [9]:
from triqs_tprf.lattice import chiq_sum_nu, chiq_sum_nu_q
p.chi_w = chiq_sum_nu_q(chi_wk) # static suscept
p.chi_wk = chiq_sum_nu(chi_wk) # static suscept